# 使用 k 近邻算法改进网站的配对效果

In [1]:
# 导入模块
import numpy as np
import operator

In [8]:
#数据的读取
#数据说明
#特征：Percentage of time spent playing video games，Number of frequent flyer miles earned per year， Liters of ice cream consumed weekly
#标签：not at all','in small doses', 'in large doses'
def read_data(filename):
    result_dictionary = {'largeDoses':3,'smallDoses':2,'notLike':1}
    fr = open(filename)
    arrayLines = fr.readlines()
    numberOfLines = len(arrayLines)
    dataMax = np.zeros((numberOfLines,3))
    classLabelVector = []
    index = 0
    for line in arrayLines :
        #除去字符串的首尾空格
        line = line.strip()
        listFormLine = line.split('\t')
        dataMax[index,:] = listFormLine[0:3]
        if(listFormLine[-1].isdigit()):
            classLabelVector.append(int(listFormLine[-1]))
        else:
            classLabelVector.append(result_dictionary.get(listFormLine[-1]))
        index += 1
    return dataMax,classLabelVector

In [13]:
#特征归一化
#(x-x_min)/(x_max - x_min)
def autoNorm(dataSet):
    min = dataSet.min(0)
    max = dataSet.max(0)
    r = max-min
    normDataSet = np.zeros(np.shape(dataSet))
    m = dataSet.shape[0]
    #normDataSet = dataSet - np.tile(min,(m,1))
    #normDataSet = normDataSet / np.tile(r,(m,1))
    #广播
    normDataSet = (dataSet - min)/r
    return normDataSet
normDataSet = autoNorm(dataMax)
print(normDataSet)


[[0.44832535 0.39805139 0.56233353]
 [0.15873259 0.34195467 0.98724416]
 [0.28542943 0.06892523 0.47449629]
 ...
 [0.29115949 0.50910294 0.51079493]
 [0.52711097 0.43665451 0.4290048 ]
 [0.47940793 0.3768091  0.78571804]]


In [20]:
#定义K临近算法
def classifyKNN(train_X,test_X,lables,k):
    diff = train_X - test_X
    sqDiff = diff **2
    sqDistance = sqDiff.sum(axis = 1)
    distance = sqDistance ** 0.5
    #获取距离由近到远的索引
    sortedDistance = distance.argsort()
    classcount={}
    for i in range(k):
        voLables = lables[sortedDistance[i]]
        classcount[voLables] = classcount.get(voLables,0)+1
    #key = operator.itemgetter(1) 按值排序
    #reverse=True 降序
    sortedClassCount = sorted(classcount.items(),key = operator.itemgetter(1),reverse=True)
    return sortedClassCount[0][0]
#作为完整程序，验证分类器
def classifyTest():
    #10%的程序用来测试
    testRatio = 0.10
    #读数据
    dataMax,labels = read_data('datingTestSet2.txt')
    #归一化
    normDataSet = autoNorm(dataMax)
    m = dataMax.shape[0]
    #测试样本数
    numTestVector = int(m * testRatio)
    #分类错误计数
    errorCount = 0
    for i in range(numTestVector):
        classifierResult = classifyKNN(normDataSet[numTestVector:m,],normDataSet[i,:],labels[numTestVector:m],3)
        if(classifierResult!=labels[i]):
            errorCount = errorCount +1
    print(errorCount)
    print("The total error rate is {number}".format(number = errorCount/float(numTestVector)))
classifyTest()

5
The total error rate is 0.05
